### Explorative Analyse der Smartmeter Daten

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from data_loader import smartmeter_load
import seaborn as sns
import matplotlib as plt
import polars as pl

In [ ]:
smart_meter = smartmeter_load(data_path ='../data/raw/households')
smart_meter.describe()

In [ ]:

smart_meter = smart_meter.with_columns(
    pl.col("timestamp_local").dt.date().alias("date")
)

smart_meter_sum = (smart_meter.group_by('date').agg([
                                                        pl.col("kwh_received_total").sum(),
                                                        pl.col("kwh_received_heatpump").sum(),
                                                        pl.col("kwh_received_other").sum(),
                                                        pl.col("kwh_returned_total").sum()
                                                        ]).sort("date"))

import matplotlib.pyplot as plt

metrics = ["kwh_received_total", "kwh_received_heatpump", "kwh_received_other", "kwh_returned_total"]
fig, axes = plt.subplots(len(metrics), 1, figsize=(12, 14), sharex=True)

for i, col in enumerate(metrics):
    # Polars Series direkt plotten
    axes[i].plot(
        smart_meter_sum["date"], 
        smart_meter_sum[col], 
        color=f"C{i}",
        linewidth=1
    )
    axes[i].set_title(f"Tägliche Summme: {col}", fontsize=12)
    axes[i].set_ylabel("kWh / Tag")
    axes[i].grid(True, alpha=0.2)

plt.xlabel("Datum")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Wir berechnen die Summen pro Gruppe und Monat für einen besseren Überblick
# Falls du 'date' noch nicht hast, vorher: .with_columns(pl.col("timestamp").dt.date().alias("date"))
plot_df = (
    smart_meter
    .with_columns(pl.col("date").dt.month().alias("month"))
    .group_by(["group_assignment", "month"])
    .agg([
        pl.col("kwh_received_total").sum(),
        pl.col("kwh_returned_total").sum()
    ])
    .sort("month")
)

# Plotten
groups = plot_df["group_assignment"].unique().to_list()
fig, axes = plt.subplots(1, len(groups), figsize=(14, 5), sharey=True)

for i, group in enumerate(groups):
    # Filter für die aktuelle Gruppe und Umwandlung zu Pandas nur für den Plot
    group_data = plot_df.filter(pl.col("group_assignment") == group).to_pandas()
    
    # Bar-Plot: Received vs Returned
    group_data.set_index("month")[["kwh_received_total", "kwh_returned_total"]].plot(
        kind="bar", 
        stacked=True, 
        ax=axes[i],
        color=["#1f77b4", "#ff7f0e"] # Blau für Bezug, Orange für Rückspeisung
    )
    
    axes[i].set_title(f"Gruppe: {group}")
    axes[i].set_xlabel("Monat")
    axes[i].set_ylabel("Gesamt kWh")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Wir berechnen die Summen pro Gruppe und Monat für einen besseren Überblick
# Falls du 'date' noch nicht hast, vorher: .with_columns(pl.col("timestamp").dt.date().alias("date"))
plot_df = (
    smart_meter
    .with_columns(pl.col("date").dt.month().alias("month"))
    .group_by(["affects_timepoint", "month"])
    .agg([
        pl.col("household_id").count(),
    ])
    .sort("month")
)

# Plotten
groups = plot_df["affects_timepoint"].unique().to_list()
fig, axes = plt.subplots(1, len(groups), figsize=(14, 5), sharey=True)

for i, group in enumerate(groups):
    # Filter für die aktuelle Gruppe und Umwandlung zu Pandas nur für den Plot
    group_data = plot_df.filter(pl.col("affects_timepoint") == group).to_pandas()
    
    # Bar-Plot: Received vs Returned
    group_data.set_index("month")[["household_id"]].plot(
        kind="bar", 
        stacked=True, 
        ax=axes[i],
        color=["#1f77b4", "#ff7f0e"] # Blau für Bezug, Orange für Rückspeisung
    )
    
    axes[i].set_title(f"Gruppe: {group}")
    axes[i].set_xlabel("Monat")
    axes[i].set_ylabel("Anzahl Haushalte")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Wir berechnen die Summen pro Gruppe und Monat für einen besseren Überblick
# Falls du 'date' noch nicht hast, vorher: .with_columns(pl.col("timestamp").dt.date().alias("date"))
plot_df = (
    smart_meter
    .with_columns(pl.col("date").dt.month().alias("month"))
    .group_by(["affects_timepoint", "month"])
    .agg([
        pl.col("kwh_received_total").sum(),
        pl.col("kwh_returned_total").sum()
    ])
    .sort("month")
)

# Plotten
groups = plot_df["affects_timepoint"].unique().to_list()
fig, axes = plt.subplots(1, len(groups), figsize=(14, 5), sharey=True)

for i, group in enumerate(groups):
    # Filter für die aktuelle Gruppe und Umwandlung zu Pandas nur für den Plot
    group_data = plot_df.filter(pl.col("affects_timepoint") == group).to_pandas()
    
    # Bar-Plot: Received vs Returned
    group_data.set_index("month")[["kwh_received_total", "kwh_returned_total"]].plot(
        kind="bar", 
        stacked=True, 
        ax=axes[i],
        color=["#1f77b4", "#ff7f0e"] # Blau für Bezug, Orange für Rückspeisung
    )
    
    axes[i].set_title(f"Gruppe: {group}")
    axes[i].set_xlabel("Monat")
    axes[i].set_ylabel("Gesamt kWh")

plt.tight_layout()
plt.show()

## Explorative Analyse der Wetterdaten

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from data_loader import  weather_load

weather_data = weather_load(data_path ='../data/raw/weather')
weather_data.describe()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import polars as pl

# Deine Metriken
metrics = [
    "temperature_avg_daily", "temperature_max_daily", "temperature_min_daily", 
    "heatingdegree_sia_daily", "heatingdegree_us_daily", "coolingdegree_us_daily",
    "humidity_avg_daily", "precipitation_total_daily", "sunshine_duration_daily"
]

fig, axes = plt.subplots(len(metrics), 1, figsize=(12, 22), sharex=True)

for i, col in enumerate(metrics):
    # 1. Daten plotten
    axes[i].plot(
        weather_data["date"], 
        weather_data[col], 
        color=f"C{i}",
        linewidth=0.8,
        label=col
    )

    # 2. X-Achse präzise einstellen
    # Große Marker für die Jahre
    axes[i].xaxis.set_major_locator(mdates.YearLocator())
    axes[i].xaxis.set_major_formatter(mdates.DateFormatter('\n%Y')) # Jahr in neuer Zeile
    
    # Spezifische Monate: März(3), Juni(6), Sept(9), Dez(12)
    axes[i].xaxis.set_minor_locator(mdates.MonthLocator(bymonth=[3, 6, 9, 12]))
    axes[i].xaxis.set_minor_formatter(mdates.DateFormatter('%b')) # %b = Monatskürzel (Mar, Jun, Sep...)

    # 3. Optik: Gitter für die Monate aktivieren
    axes[i].grid(True, which='minor', axis='x', color='gray', linestyle='--', alpha=0.3)
    axes[i].grid(True, which='major', axis='y', alpha=0.2)

    # Styling
    axes[i].set_title(f"Verlauf: {col}", fontsize=10, loc='left')
    axes[i].tick_params(axis='x', which='minor', labelsize=8, rotation=0)

plt.xlabel("Zeitachse (Quartalsmarkierungen)")
plt.tight_layout()
plt.show()

# Explorative Analyse der Technischen Haushaltsdaten

In [ ]:
import sys
import os

# Wir gehen eine Ebene höher zum Projekt-Root und fügen 'src' hinzu
sys.path.append(os.path.abspath("../src"))

# Jetzt sollte der Import klappen
from data_loader import household_load

household_info = household_load(data_path ='../data/raw/households_info/')
household_info.describe()

In [ ]:
household_info.schema

In [ ]:
import polars as pl
import matplotlib.pyplot as plt

metrics = [
    "installation_haspvsystem", "protocols_available", "protocols_hasmultiplevisits", 
    "metadata_available", "smartmeterdata_available_15min", 
    "smartmeterdata_available_daily", "smartmeterdata_available_monthly"
]

fig, axes = plt.subplots(len(metrics), 1, figsize=(8, 30))

for i, col in enumerate(metrics):
    # Wir greifen direkt auf die Spalte zu (Series), füllen nulls und zählen dann
    counts = (
        household_info[col]
        .fill_null("Unbekannt")
        .value_counts()
        .sort(by="count", descending=True)
    )
    
    # Da counts nun ein DataFrame mit der Spalte 'col' und 'count' ist:
    x_labels = counts[col].cast(pl.String).to_list()
    y_values = counts["count"].to_list()
    
    axes[i].bar(x_labels, y_values, color="skyblue", edgecolor="black")
    
    axes[i].set_title(f"Verteilung: {col}", fontsize=12, fontweight='bold')
    axes[i].set_ylabel("Anzahl Haushalte")
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Explorative Analyse der Meta Informationen zum Haushalt

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

from data_loader import household_metainfo_load

household_info = household_metainfo_load(data_path ='../data/raw/protocols/')
household_info.describe()

In [ ]:
import polars as pl
import matplotlib.pyplot as plt

# 1. Doppeltes Komma entfernt und Metriken definiert
metrics = [ 'building_type', 'building_constructionyear_interval', 'heatpump_installation_type', 'heatpump_installation_manufacturer', 'heatpump_installation_model', 'heatpump_installation_refrigerant_type', 'heatpump_installation_normpoint', 'heatpump_installation_normpoint_heatingpower', 'heatpump_installation_location', 'dhw_byheatpump_timeinterval', 'dhw_production_typeofheating', 'heatpump_electricityconsumption_categorization', 'heatpump_installation_incorrectlyplanned_categorization', 'dhw_temperaturesetting_categorization', 'heatdistribution_expansiontank_pressure_categorization'
]

# Erhöhte Breite und Höhe pro Subplot für bessere Lesbarkeit
fig, axes = plt.subplots(len(metrics), 1, figsize=(12, len(metrics) * 5))

for i, col in enumerate(metrics):
    counts = (
        household_info[col]
        .fill_null("Unbekannt")
        .value_counts()
        .sort(by="count", descending=True)
    )
    
    x_labels = counts[col].cast(pl.String).to_list()
    y_values = counts["count"].to_list()
    
    bars = axes[i].bar(x_labels, y_values, color="skyblue", edgecolor="black", alpha=0.8)
    
    # --- TITEL ANPASSUNG ---
    # 'pad' schiebt den Titel nach oben, weg vom Plot
    axes[i].set_title(f"Verteilung: {col}", fontsize=16, fontweight='bold', pad=25)
    
    axes[i].set_ylabel("Anzahl Haushalte", fontsize=12)
    axes[i].grid(axis='y', linestyle='--', alpha=0.5)
    
    # --- X-ACHSE ANPASSUNG ---
    axes[i].set_xticks(range(len(x_labels)))
    # rotation_mode='anchor' sorgt für eine saubere Ausrichtung der gedrehten Texte
    axes[i].set_xticklabels(x_labels, rotation=45, ha='right', fontsize=11, rotation_mode='anchor')
    
    # Werte über den Balken anzeigen
    axes[i].bar_label(bars, padding=3, fontsize=10)

# 'hspace' steuert den vertikalen Abstand zwischen den Subplots
# 'top' lässt oben Platz für den ersten Titel
plt.subplots_adjust(hspace=0.8, top=0.95)

# Tight_layout mit zusätzlichem Padding
plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
import polars.selectors as cs
float_cols = household_info.select(cs.float()).columns

print(float_cols)

In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# Deine Liste (ich habe die unnötige Verschachtelung der Liste korrigiert)
metrics = [
    'building_housingunits', 'building_constructionyear', 'building_floorareaheated_total', 
    'building_pvsystem_size', 'heatpump_installation_heatingcapacity', 
    'heatpump_installation_normpoint_cop', 'dhw_temperaturesetting_beforevisit',
    'heatdistribution_expansiontank_pressure_actual'
] 
# Tipp: Ich habe hier nur eine Auswahl genommen, du kannst deine ganze Liste oben einfügen.

# Wir nutzen Seaborn für schönere statistische Plots
sns.set_theme(style="whitegrid")

# Dynamische Erstellung der Subplots
fig, axes = plt.subplots(len(metrics), 1, figsize=(12, len(metrics) * 6))

# Falls nur eine Metrik vorhanden ist, muss axes in eine Liste umgewandelt werden
if len(metrics) == 1:
    axes = [axes]

for i, col in enumerate(metrics):
    # Daten vorbereiten: Null-Werte entfernen, da Histogramme sonst Fehler werfen können
    data = household_info.select(col).drop_nulls().to_series()
    
    if data.len() == 0:
        axes[i].text(0.5, 0.5, f"Keine Daten für {col}", ha='center')
        continue

    # Histogramm mit Kerndichteschätzung (KDE) für die Verteilungskurve
    sns.histplot(data, bins=30, kde=True, ax=axes[i], color="skyblue", edgecolor="black")
    
    # --- TITEL & LABELS ---
    axes[i].set_title(f"Verteilung von: {col}", fontsize=16, fontweight='bold', pad=20)
    axes[i].set_xlabel("Wert", fontsize=12)
    axes[i].set_ylabel("Anzahl Haushalte", fontsize=12)
    
    # Statistische Kennzahlen als Linien einzeichnen (Mittelwert & Median)
    mean_val = data.mean()
    median_val = data.median()
    axes[i].axvline(mean_val, color='red', linestyle='--', label=f'Schnitt: {mean_val:.2f}')
    axes[i].axvline(median_val, color='green', linestyle='-', label=f'Median: {median_val:.2f}')
    axes[i].legend()

# Layout optimieren
plt.tight_layout(pad=4.0)
plt.show()